In [1]:
import numpy as np
import hashlib
import json
import zipfile
import os
import glob
import shutil
import random
print('Run this cell to ensure you have all the packages you need for the following program. Put the zip file full of txt in the same folder as this program.')

Run this cell to ensure you have all the packages you need for the following program. Put the zip file full of txt in the same folder as this program.


In [31]:
def first_and_last_name_list(first_name_csv_file, last_name_csv_file):
    first_name_array = []
    last_name_array = []
    with open(first_name_csv_file) as f:
        lines = f.readlines()
    lines = np.array(lines)
    for rows in lines:
        first_name_array.append(rows)
    first_name_array = np.array(first_name_array)
    with open(last_name_csv_file) as f:
        lines = f.readlines()
    lines = np.array(lines)
    for rows in lines:
        last_name_array.append(rows)
    last_name_array = np.array(last_name_array)
    return first_name_array, last_name_array

In [32]:
def dictionary_initialization_chat_and_transcript(csv_file):
    name_onyen_dictionary = {}
    with open(csv_file) as f:
        next(f)
        next(f)
        next(f)
        lines = f.readlines()
    lines = np.array(lines)
    for rows in lines:
        if (rows.split(",")[2] == ""): # No last name
            name_onyen_dictionary.update({rows.split(",")[3]:rows.split(",")[1]})
        if (rows.split(",")[3] == ""): # No first name
            name_onyen_dictionary.update({rows.split(",")[2]:rows.split(",")[1]})
        name_onyen_dictionary.update({rows.split(",")[3] + " " + rows.split(",")[2]:rows.split(",")[1]}) # Both first and last name
    # print(name_onyen_dictionary)
    return name_onyen_dictionary

In [37]:
def extract_and_hash_chat(txt_file, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file):
    # Get updated first and last name list
    first_name_array, last_name_array = first_and_last_name_list(first_name_csv_file, last_name_csv_file)
    # Get updated general anti_hashmap
    with open(general_first_name_hashmap_json_file) as f:
        general_first_name_hashmap = json.load(f)
    with open(general_last_name_hashmap_json_file) as f:
        general_last_name_hashmap = json.load(f)
    with open(general_onyen_hashmap_json_file) as f:
        general_onyen_hashmap = json.load(f)
    # Establish the name-onyen dictionary
    name_onyen_dictionary = dictionary_initialization_chat_and_transcript(csv_file)
    hashed_first_names = ()
    hashed_last_names = ()
    hashed_onyen = ()
    fake_first_name_hashed_value_dictionary = {}
    fake_last_name_hashed_value_dictionary = {}
    fake_onyen_hashed_value_dictionary = {}
    with open(txt_file,encoding='utf-8') as f:
        lines = f.readlines()
    lines = np.array(lines)
    answer = []
    fake_name_answer = []
    hash_map = {}
    anti_hash_map = {}
    for rows in lines:
        # print("Original message: " + rows)
        if "From" not in rows:
            line = rows.split('\t')
            if len(line)<2:
                continue
            name = line[1]
            if (len(name.split(" ")) < 2): # No first or last name
                first_name = name.split(" ")[0]
                last_name = ""
            else: # 这里存在一个问题，如果一个人的名字有不止一个空格（如middle name），将会导致first name与last name的选取出错
                first_name = name.split(" ")[0]
                last_name = name.split(" ")[1]

            # Using the dictionary to convert the names to corresponding onyens
            if name not in name_onyen_dictionary.keys():
                onyen = '*'+name+'*'
            else:
                onyen = name_onyen_dictionary[name]

            # sha-256 encryption
            if (first_name != ""):
                encoded_first = first_name.encode()
                result_first = hashlib.sha256(encoded_first)
                if (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # First time that encounters a first name
                    hashed_first_names = hashed_first_names + (result_first.hexdigest(),)
                    fake_first_name = first_name_array[0].replace("\n", "") # Always pick the first element
                    first_name_array = np.delete(first_name_array,0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                    fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                    general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                elif (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() in general_first_name_hashmap): # NOT first time that encounters a first name
                    fake_first_name = general_first_name_hashmap[result_first.hexdigest()]
                    fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                elif (result_first.hexdigest() in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # NOT first time that encounters a first name
                    fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
                    general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                else:
                    fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
            else: # First name is ""
                fake_first_name = ""

            if (last_name != ""):
                encoded_last = last_name.encode()
                result_last = hashlib.sha256(encoded_last)
                if (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # First time that encounters a last name
                    hashed_last_names = hashed_last_names + (result_last.hexdigest(),)
                    fake_last_name = last_name_array[0].replace("\n", "") # Always pick the first element
                    last_name_array = np.delete(last_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                    fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                    general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                elif (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() in general_last_name_hashmap): # NOT first time that encounters a last name
                    fake_last_name = general_last_name_hashmap[result_last.hexdigest()]
                    fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                elif (result_last.hexdigest() in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # NOT first time that encounters a last name
                    fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
                    general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                else:
                    fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
            else: # Last name is ""
                fake_last_name = ""

            if (onyen != ""):
                encoded_onyen = onyen.encode()
                result_onyen = hashlib.sha256(encoded_onyen)
                if (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # First time that encounters an onyen
                    hashed_onyen = hashed_onyen + (result_onyen.hexdigest(),)
                    if (fake_first_name != "") and (fake_last_name != ""):
                        fake_onyen = fake_first_name + fake_last_name # 这里暂时不处理同名同姓的不同人的问题，因为如果两人名字相同但onyen不同，在Zoom由name map到onyen时就会报错
                    elif (fake_first_name == "") and (fake_last_name != ""):
                        fake_onyen = fake_last_name + fake_last_name
                    else:
                        fake_onyen = fake_first_name + fake_first_name
                    fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                    general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                elif (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() in general_onyen_hashmap): # NOT first time that encounters an onyen
                    fake_onyen = general_onyen_hashmap[result_onyen.hexdigest()]
                    fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                elif (result_onyen.hexdigest() in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # NOT first time that encounters an onyen
                    fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
                    general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                else:
                    fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
            else: # Onyen is ""
                fake_onyen = ""

            #hashed_str = rows[:idx1 + len(sub1) + 1]
            #anti_hashed_str = rows[:idx1 + len(sub1) + 1]
            name_onyen_substring = ""
            anti_name_onyen_substring = ""

            if (first_name != ""):
                name_onyen_substring = name_onyen_substring + str(result_first.hexdigest())
                anti_name_onyen_substring = anti_name_onyen_substring + fake_first_name
            if ((first_name != "") and (last_name != "")):
                name_onyen_substring = name_onyen_substring + " "
                anti_name_onyen_substring = anti_name_onyen_substring + " "
            if (last_name != ""):
                name_onyen_substring = name_onyen_substring + str(result_last.hexdigest())
                anti_name_onyen_substring = anti_name_onyen_substring + fake_last_name
            if (onyen != ""):
                name_onyen_substring = name_onyen_substring + " " + "(" + str(result_onyen.hexdigest()) + ")"
                anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + fake_onyen + ")"
            else:
                name_onyen_substring = name_onyen_substring + " " + "(" + ")"
                anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + ")"

            hashed_str = line[0] +'\t' + name_onyen_substring  +':\t' +line[2]
            anti_hashed_str = line[0] + '\t' + anti_name_onyen_substring + ':\t' + line[2]

            # print("Hashed message: " + hashed_str)
            answer.append(hashed_str)
            fake_name_answer.append(anti_hashed_str)
            hash_map.update({name + " (" + onyen + ")":name_onyen_substring})
            if (first_name != "") and (last_name != "") and (onyen != ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (first_name == "") and (onyen == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " ()"})
            elif (last_name != "") and (onyen != ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " ()"})
            elif (first_name == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (last_name == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (onyen == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " ()"})
        elif ("From" in rows) and (":" in rows):
            if "(Direct Message)" not in rows:
                # initializing substrings
                sub1 = "From"
                sub2 = " :"
                
                # getting index of substrings
                idx1 = rows.index(sub1)
                idx2 = rows.index(sub2)

                # length of substring 1 is added to
                # get string from next character
                name = rows[idx1 + len(sub1) + 1: idx2].lstrip().rstrip() # remove spaces
                if (len(name.split(" ")) < 2): # No first or last name
                    first_name = name.split(" ")[0]
                    last_name = ""
                else: # 这里存在一个问题，如果一个人的名字有不止一个空格（如middle name），将会导致first name与last name的选取出错
                    first_name = name.split(" ")[0]
                    last_name = name.split(" ")[1]
                
                # Using the dictionary to convert the names to corresponding onyens
                if name not in name_onyen_dictionary.keys():
                    onyen = '*'+name+'*'
                else:
                    onyen = name_onyen_dictionary[name]

                # sha-256 encryption
                if (first_name != ""):
                    encoded_first = first_name.encode()
                    result_first = hashlib.sha256(encoded_first)
                    if (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # First time that encounters a first name
                        hashed_first_names = hashed_first_names + (result_first.hexdigest(),)
                        fake_first_name = first_name_array[0].replace("\n", "") # Always pick the first element
                        first_name_array = np.delete(first_name_array,0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                        general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                    elif (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() in general_first_name_hashmap): # NOT first time that encounters a first name
                        fake_first_name = general_first_name_hashmap[result_first.hexdigest()]
                        fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                    elif (result_first.hexdigest() in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # NOT first time that encounters a first name
                        fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
                        general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                    else:
                        fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
                else: # First name is ""
                    fake_first_name = ""
                        
                if (last_name != ""):
                    encoded_last = last_name.encode()
                    result_last = hashlib.sha256(encoded_last)
                    if (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # First time that encounters a last name
                        hashed_last_names = hashed_last_names + (result_last.hexdigest(),)
                        fake_last_name = last_name_array[0].replace("\n", "") # Always pick the first element
                        last_name_array = np.delete(last_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                        general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                    elif (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() in general_last_name_hashmap): # NOT first time that encounters a last name
                        fake_last_name = general_last_name_hashmap[result_last.hexdigest()]
                        fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                    elif (result_last.hexdigest() in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # NOT first time that encounters a last name
                        fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
                        general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                    else:
                        fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
                else: # Last name is ""
                    fake_last_name = ""
                
                if (onyen != ""):
                    encoded_onyen = onyen.encode()
                    result_onyen = hashlib.sha256(encoded_onyen)
                    if (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # First time that encounters an onyen
                        hashed_onyen = hashed_onyen + (result_onyen.hexdigest(),)
                        if (fake_first_name != "") and (fake_last_name != ""):
                            fake_onyen = fake_first_name + fake_last_name # 这里暂时不处理同名同姓的不同人的问题，因为如果两人名字相同但onyen不同，在Zoom由name map到onyen时就会报错
                        elif (fake_first_name == "") and (fake_last_name != ""):
                            fake_onyen = fake_last_name + fake_last_name
                        else:
                            fake_onyen = fake_first_name + fake_first_name
                        fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                        general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                    elif (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() in general_onyen_hashmap): # NOT first time that encounters an onyen
                        fake_onyen = general_onyen_hashmap[result_onyen.hexdigest()]
                        fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                    elif (result_onyen.hexdigest() in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # NOT first time that encounters an onyen
                        fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
                        general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                    else:
                        fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
                else: # Onyen is ""
                    fake_onyen = ""
                    
                hashed_str = rows[:idx1 + len(sub1) + 1]
                anti_hashed_str = rows[:idx1 + len(sub1) + 1]
                name_onyen_substring = ""
                anti_name_onyen_substring = ""
                
                if (first_name != ""):
                    name_onyen_substring = name_onyen_substring + str(result_first.hexdigest())
                    anti_name_onyen_substring = anti_name_onyen_substring + fake_first_name
                if ((first_name != "") and (last_name != "")):
                    name_onyen_substring = name_onyen_substring + " "
                    anti_name_onyen_substring = anti_name_onyen_substring + " "
                if (last_name != ""):
                    name_onyen_substring = name_onyen_substring + str(result_last.hexdigest())
                    anti_name_onyen_substring = anti_name_onyen_substring + fake_last_name
                if (onyen != ""):
                    name_onyen_substring = name_onyen_substring + " " + "(" + str(result_onyen.hexdigest()) + ")"
                    anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + fake_onyen + ")"
                else:
                    name_onyen_substring = name_onyen_substring + " " + "(" + ")"
                    anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + ")"
                    
                hashed_str = hashed_str + name_onyen_substring + rows[idx2:]
                anti_hashed_str = anti_hashed_str + anti_name_onyen_substring + rows[idx2:]

                # print("Hashed message: " + hashed_str)
                answer.append(hashed_str)
                fake_name_answer.append(anti_hashed_str)
                hash_map.update({name + " (" + onyen + ")":name_onyen_substring})
                if (first_name != "") and (last_name != "") and (onyen != ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
                elif (first_name == "") and (onyen == ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " ()"})
                elif (last_name != "") and (onyen != ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " ()"})
                elif (first_name == ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
                elif (last_name == ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
                elif (onyen == ""):
                    anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " ()"})
                    
            if "(Direct Message)" in rows:
                # initializing substrings
                sub1 = "From"
                sub2 = "to"
                sub3 = "to"
                sub4 = "(Direct Message) :"

                # getting index of substrings
                idx1 = rows.index(sub1)
                idx2 = rows.index(sub2)
                idx3 = rows.index(sub3)
                idx4 = rows.index(sub4)

                # length of substring 1 is added to
                # get string from next character
                name1 = rows[idx1 + len(sub1) + 1: idx2].lstrip().rstrip() # remove spaces
                if (len(name1.split(" ")) < 2): # No first or last name
                    first_name1 = name1.split(" ")[0]
                    last_name1 = ""
                else: # 这里存在一个问题，如果一个人的名字有不止一个空格（如middle name），将会导致first name与last name的选取出错
                    first_name1 = name1.split(" ")[0]
                    last_name1 = name1.split(" ")[1]
                name2 = rows[idx3 + len(sub3) + 1: idx4].lstrip().rstrip() # remove spaces
                if (len(name2.split(" ")) < 2): # No first or last name
                    first_name2 = name2.split(" ")[0]
                    last_name2 = ""
                else: # 这里存在一个问题，如果一个人的名字有不止一个空格（如middle name），将会导致first name与last name的选取出错
                    first_name2 = name2.split(" ")[0]
                    last_name2 = name2.split(" ")[1]
                
                # Using the dictionary to convert the names to corresponding onyens
                if name1 not in name_onyen_dictionary.keys():
                    onyen1 = '*'+name1+'*'
                else: 
                    onyen1 = name_onyen_dictionary[name1]
                if name2 not in name_onyen_dictionary.keys():
                    onyen2 = '*'+name2+'*'
                else:
                    onyen2 = name_onyen_dictionary[name2]

                # sha-256 encryption
                if (first_name1 != ""):
                    encoded_first1 = first_name1.encode()
                    result_first1 = hashlib.sha256(encoded_first1)
                    if (result_first1.hexdigest() not in hashed_first_names) and (result_first1.hexdigest() not in general_first_name_hashmap): # First time that encounters a first name
                        hashed_first_names = hashed_first_names + (result_first1.hexdigest(),)
                        fake_first_name1 = first_name_array[0].replace("\n", "") # Always pick the first element
                        first_name_array = np.delete(first_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_first_name_hashed_value_dictionary.update({result_first1.hexdigest():fake_first_name1})
                        general_first_name_hashmap.update({result_first1.hexdigest():fake_first_name1})
                    elif (result_first1.hexdigest() not in hashed_first_names) and (result_first1.hexdigest() in general_first_name_hashmap): # NOT first time that encounters a first name
                        fake_first_name1 = general_first_name_hashmap[result_first1.hexdigest()]
                        fake_first_name_hashed_value_dictionary.update({result_first1.hexdigest():fake_first_name1})
                    elif (result_first1.hexdigest() in hashed_first_names) and (result_first1.hexdigest() not in general_first_name_hashmap): 
                        fake_first_name1 = fake_first_name_hashed_value_dictionary[result_first1.hexdigest()]
                        general_first_name_hashmap.update({result_first1.hexdigest():fake_first_name1})
                    else:
                        fake_first_name1 = fake_first_name_hashed_value_dictionary[result_first1.hexdigest()]
                else: # First name is ""
                    fake_first_name1 = ""
                        
                if (last_name1 != ""):
                    encoded_last1 = last_name1.encode()
                    result_last1 = hashlib.sha256(encoded_last1)
                    if (result_last1.hexdigest() not in hashed_last_names) and (result_last1.hexdigest() not in general_last_name_hashmap): # First time that encounters a last name
                        hashed_last_names = hashed_last_names + (result_last1.hexdigest(),)
                        fake_last_name1 = last_name_array[0].replace("\n", "") # Always pick the first element
                        last_name_array = np.delete(last_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_last_name_hashed_value_dictionary.update({result_last1.hexdigest():fake_last_name1})
                        general_last_name_hashmap.update({result_last1.hexdigest():fake_last_name1})
                    elif (result_last1.hexdigest() not in hashed_last_names) and (result_last1.hexdigest() in general_last_name_hashmap): # NOT first time that encounters a last name
                        fake_last_name1 = general_last_name_hashmap[result_last1.hexdigest()]
                        fake_last_name_hashed_value_dictionary.update({result_last1.hexdigest():fake_last_name1})
                    elif (result_last1.hexdigest() in hashed_last_names) and (result_last1.hexdigest() not in general_last_name_hashmap):
                        fake_last_name1 = fake_last_name_hashed_value_dictionary[result_last1.hexdigest()]
                        general_last_name_hashmap.update({result_last1.hexdigest():fake_last_name1})
                    else:
                        fake_last_name1 = fake_last_name_hashed_value_dictionary[result_last1.hexdigest()]
                else: # Last name is ""
                    fake_last_name1 = ""
                
                if (onyen1 != ""):
                    encoded_onyen1 = onyen1.encode()
                    result_onyen1 = hashlib.sha256(encoded_onyen1)
                    if (result_onyen1.hexdigest() not in hashed_onyen) and (result_onyen1.hexdigest() not in general_onyen_hashmap): # First time that encounters an onyen
                        hashed_onyen = hashed_onyen + (result_onyen1.hexdigest(),)
                        if (fake_first_name1 != "") and (fake_last_name1 != ""):
                            fake_onyen1 = fake_first_name1 + fake_last_name1 # 这里暂时不处理同名同姓的不同人的问题，因为如果两人名字相同但onyen不同，在Zoom由name map到onyen时就会报错
                        elif (fake_first_name1 == "") and (fake_last_name1 != ""):
                            fake_onyen1 = fake_last_name1 + fake_last_name1
                        else:
                            fake_onyen1 = fake_first_name1 + fake_first_name1
                        fake_onyen_hashed_value_dictionary.update({result_onyen1.hexdigest():fake_onyen1})
                        general_onyen_hashmap.update({result_onyen1.hexdigest():fake_onyen1})
                    elif (result_onyen1.hexdigest() not in hashed_onyen) and (result_onyen1.hexdigest() in general_onyen_hashmap): # NOT first time that encounters an onyen
                        fake_onyen1 = general_onyen_hashmap[result_onyen1.hexdigest()]
                        fake_onyen_hashed_value_dictionary.update({result_onyen1.hexdigest():fake_onyen1})
                    elif (result_onyen1.hexdigest() in hashed_onyen) and (result_onyen1.hexdigest() not in general_onyen_hashmap):
                        fake_onyen1 = fake_onyen_hashed_value_dictionary[result_onyen1.hexdigest()]
                        general_onyen_hashmap.update({result_onyen1.hexdigest():fake_onyen1})
                    else:
                        fake_onyen1 = fake_onyen_hashed_value_dictionary[result_onyen1.hexdigest()]
                else: # Onyen is ""
                    fake_onyen1 = ""
                    
                if (first_name2 != ""):
                    encoded_first2 = first_name2.encode()
                    result_first2 = hashlib.sha256(encoded_first2)
                    if (result_first2.hexdigest() not in hashed_first_names) and (result_first2.hexdigest() not in general_first_name_hashmap): # First time that encounters a first name
                        hashed_first_names = hashed_first_names + (result_first2.hexdigest(),)
                        fake_first_name2 = first_name_array[0].replace("\n", "") # Always pick the first element
                        first_name_array = np.delete(first_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_first_name_hashed_value_dictionary.update({result_first2.hexdigest():fake_first_name2})
                        general_first_name_hashmap.update({result_first2.hexdigest():fake_first_name2})
                    elif (result_first2.hexdigest() not in hashed_first_names) and (result_first2.hexdigest() in general_first_name_hashmap): # NOT first time that encounters a first name
                        fake_first_name2 = general_first_name_hashmap[result_first2.hexdigest()]
                        fake_first_name_hashed_value_dictionary.update({result_first2.hexdigest():fake_first_name2})
                    elif (result_first2.hexdigest() in hashed_first_names) and (result_first2.hexdigest() not in general_first_name_hashmap): 
                        fake_first_name2 = fake_first_name_hashed_value_dictionary[result_first2.hexdigest()]
                        general_first_name_hashmap.update({result_first2.hexdigest():fake_first_name2})
                    else:
                        fake_first_name2 = fake_first_name_hashed_value_dictionary[result_first2.hexdigest()]
                else: # First name is ""
                    fake_first_name2 = ""
                        
                if (last_name2 != ""):
                    encoded_last2 = last_name2.encode()
                    result_last2 = hashlib.sha256(encoded_last2)
                    if (result_last2.hexdigest() not in hashed_last_names) and (result_last2.hexdigest() not in general_last_name_hashmap): # First time that encounters a last name
                        hashed_last_names = hashed_last_names + (result_last2.hexdigest(),)
                        fake_last_name2 = last_name_array[0].replace("\n", "") # Always pick the first element
                        last_name_array = np.delete(last_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                        fake_last_name_hashed_value_dictionary.update({result_last2.hexdigest():fake_last_name2})
                        general_last_name_hashmap.update({result_last2.hexdigest():fake_last_name2})
                    elif (result_last2.hexdigest() not in hashed_last_names) and (result_last2.hexdigest() in general_last_name_hashmap): # NOT first time that encounters a last name
                        fake_last_name2 = general_last_name_hashmap[result_last2.hexdigest()]
                        fake_last_name_hashed_value_dictionary.update({result_last2.hexdigest():fake_last_name2})
                    elif (result_last2.hexdigest() in hashed_last_names) and (result_last2.hexdigest() not in general_last_name_hashmap):
                        fake_last_name2 = fake_last_name_hashed_value_dictionary[result_last2.hexdigest()]
                        general_last_name_hashmap.update({result_last2.hexdigest():fake_last_name2})
                    else:
                        fake_last_name2 = fake_last_name_hashed_value_dictionary[result_last2.hexdigest()]
                else: # Last name is ""
                    fake_last_name2 = ""
                
                if (onyen2 != ""):
                    encoded_onyen2 = onyen2.encode()
                    result_onyen2 = hashlib.sha256(encoded_onyen2)
                    if (result_onyen2.hexdigest() not in hashed_onyen) and (result_onyen2.hexdigest() not in general_onyen_hashmap): # First time that encounters an onyen
                        hashed_onyen = hashed_onyen + (result_onyen2.hexdigest(),)
                        if (fake_first_name2 != "") and (fake_last_name2 != ""):
                            fake_onyen2 = fake_first_name2 + fake_last_name2 # 这里暂时不处理同名同姓的不同人的问题，因为如果两人名字相同但onyen不同，在Zoom由name map到onyen时就会报错
                        elif (fake_first_name2 == "") and (fake_last_name2 != ""):
                            fake_onyen2 = fake_last_name2 + fake_last_name2
                        else:
                            fake_onyen2 = fake_first_name2 + fake_first_name2
                        fake_onyen_hashed_value_dictionary.update({result_onyen2.hexdigest():fake_onyen2})
                        general_onyen_hashmap.update({result_onyen2.hexdigest():fake_onyen2})
                    elif (result_onyen2.hexdigest() not in hashed_onyen) and (result_onyen2.hexdigest() in general_onyen_hashmap): # NOT first time that encounters an onyen
                        fake_onyen2 = general_onyen_hashmap[result_onyen2.hexdigest()]
                        fake_onyen_hashed_value_dictionary.update({result_onyen2.hexdigest():fake_onyen2})
                    elif (result_onyen2.hexdigest() in hashed_onyen) and (result_onyen2.hexdigest() not in general_onyen_hashmap):
                        fake_onyen2 = fake_onyen_hashed_value_dictionary[result_onyen2.hexdigest()]
                        general_onyen_hashmap.update({result_onyen2.hexdigest():fake_onyen2})
                    else:
                        fake_onyen2 = fake_onyen_hashed_value_dictionary[result_onyen2.hexdigest()]
                else: # Onyen is ""
                    fake_onyen2 = ""
                    
                hashed_str = rows[:idx1 + len(sub1) + 1]
                anti_hashed_str = rows[:idx1 + len(sub1) + 1]
                name_onyen_substring1 = ""
                anti_name_onyen_substring1 = ""
                name_onyen_substring2 = ""
                anti_name_onyen_substring2 = ""
                
                if (first_name1 != ""):
                    name_onyen_substring1 = name_onyen_substring1 + str(result_first1.hexdigest())
                    anti_name_onyen_substring1  = anti_name_onyen_substring1 + fake_first_name1
                if ((first_name1 != "") and (last_name1 != "")):
                    name_onyen_substring1 = name_onyen_substring1 + " "
                    anti_name_onyen_substring1 = anti_name_onyen_substring1 + " "
                if (last_name1 != ""):
                    name_onyen_substring1 = name_onyen_substring1 + str(result_last1.hexdigest())
                    anti_name_onyen_substring1 = anti_name_onyen_substring1 + fake_last_name1
                if (onyen1 != ""):
                    name_onyen_substring1 = name_onyen_substring1 + " " + "(" + str(result_onyen1.hexdigest()) + ")"
                    anti_name_onyen_substring1 = anti_name_onyen_substring1 + " " + "(" + fake_onyen1 + ")"
                else:
                    name_onyen_substring1 = name_onyen_substring1 + " " + "(" + ")"
                    anti_name_onyen_substring1 = anti_name_onyen_substring1 + " " + "(" + ")"
                    
                hashed_str = hashed_str + name_onyen_substring1 + " " + rows[idx2:idx3 + len(sub3)] + " "
                anti_hashed_str = anti_hashed_str + anti_name_onyen_substring1 + " " + rows[idx2:idx3 + len(sub3)] + " "
                    
                if (first_name2 != ""):
                    name_onyen_substring2 = name_onyen_substring2 + str(result_first2.hexdigest())
                    anti_name_onyen_substring2 = anti_name_onyen_substring2 + fake_first_name2
                if ((first_name2 != "") and (last_name2 != "")):
                    name_onyen_substring2 = name_onyen_substring2 + " "
                    anti_name_onyen_substring2 = anti_name_onyen_substring2 + " "
                if (last_name2 != ""):
                    name_onyen_substring2 = name_onyen_substring2 + str(result_last2.hexdigest())
                    anti_name_onyen_substring2 = anti_name_onyen_substring2 + fake_last_name2
                if (onyen2 != ""):
                    name_onyen_substring2 = name_onyen_substring2 + " " + "(" + str(result_onyen2.hexdigest()) + ")"
                    anti_name_onyen_substring2 = anti_name_onyen_substring2 + " " + "(" + fake_onyen2 + ")"
                else:
                    name_onyen_substring2 = name_onyen_substring2 + " " + "(" + ")"
                    anti_name_onyen_substring2 = anti_name_onyen_substring2 + " " + "(" + ")"

                hashed_str = hashed_str + name_onyen_substring2 + rows[idx4:]
                anti_hashed_str = anti_hashed_str + anti_name_onyen_substring2 + rows[idx4:]
                
                # print("Hashed message: " + hashed_str)
                answer.append(hashed_str)
                fake_name_answer.append(anti_hashed_str)
                hash_map.update({name1 + " (" + onyen1 + ")":name_onyen_substring1, name2 + " (" + onyen2 + ")":name_onyen_substring2})
                if (first_name1 != "") and (last_name1 != "") and (onyen1 != ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_first1.hexdigest() + " " + result_last1.hexdigest() + " (" + result_onyen1.hexdigest() + ")"})
                elif (first_name1 == "") and (onyen1 == ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_last1.hexdigest() + " ()"})
                elif (last_name1 != "") and (onyen1 != ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_first1.hexdigest() + " ()"})
                elif (first_name1 == ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_last1.hexdigest() + " (" + result_onyen1.hexdigest() + ")"})
                elif (last_name1 == ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_first1.hexdigest() + " (" + result_onyen1.hexdigest() + ")"})
                elif (onyen1 == ""):
                    anti_hash_map.update({anti_name_onyen_substring1:result_first1.hexdigest() + " " + result_last1.hexdigest() + " ()"})
                    
                if (first_name2 != "") and (last_name2 != "") and (onyen2 != ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_first2.hexdigest() + " " + result_last2.hexdigest() + " (" + result_onyen2.hexdigest() + ")"})
                elif (first_name2 == "") and (onyen2 == ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_last2.hexdigest() + " ()"})
                elif (last_name2 != "") and (onyen2 != ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_first2.hexdigest() + " ()"})
                elif (first_name2 == ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_last2.hexdigest() + " (" + result_onyen2.hexdigest() + ")"})
                elif (last_name2 == ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_first2.hexdigest() + " (" + result_onyen2.hexdigest() + ")"})
                elif (onyen2 == ""):
                    anti_hash_map.update({anti_name_onyen_substring2:result_first2.hexdigest() + " " + result_last2.hexdigest() + " ()"})
        else:
            answer.append(rows)
            fake_name_answer.append(rows)
    
    # Save the hashed txt file
    with open(txt_file.split('.')[0] + "(hashed_chat).txt", 'w') as f:
            for line in answer:
                f.write(line)
    # Save the fake_named txt file
    with open(txt_file.split('.')[0] + "(fake_name_chat).txt", 'w') as f:
            for line in fake_name_answer:
                f.write(line)
    # Save the hashmap in json file
    with open(txt_file.split('.')[0] +'(HashTable_chat).json', 'w') as fp:
        json.dump(hash_map, fp)
    # Save the anti_hashmap in json file
    with open(txt_file.split('.')[0] +'(Anti_HashTable_chat).json', 'w') as fp:
        json.dump(anti_hash_map, fp)
    # Update fake first names.csv
    delete_file = open(first_name_csv_file, "w")
    delete_file.truncate()
    with open(first_name_csv_file, 'w') as f:
            for line in first_name_array:
                f.write(line)
    # Update fake last names.csv
    delete_file = open(last_name_csv_file, "w")
    delete_file.truncate()
    with open(last_name_csv_file, 'w') as f:
            for line in last_name_array:
                f.write(line)
    # Update the general_anti_hashmap in json file
    delete_file = open(general_first_name_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_first_name_hashmap_json_file, 'w') as fp:
        json.dump(general_first_name_hashmap, fp)
    delete_file = open(general_last_name_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_last_name_hashmap_json_file, 'w') as fp:
        json.dump(general_last_name_hashmap, fp)
    delete_file = open(general_onyen_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_onyen_hashmap_json_file, 'w') as fp:
        json.dump(general_onyen_hashmap, fp)
    return

In [38]:
def extract_and_hash_audio_transcript(vtt_file, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file):
    # Get updated first and last name list
    first_name_array, last_name_array = first_and_last_name_list(first_name_csv_file, last_name_csv_file)
    # Get updated general anti_hashmap
    with open(general_first_name_hashmap_json_file) as f:
        general_first_name_hashmap = json.load(f)
    with open(general_last_name_hashmap_json_file) as f:
        general_last_name_hashmap = json.load(f)
    with open(general_onyen_hashmap_json_file) as f:
        general_onyen_hashmap = json.load(f)
    # Establish the name-onyen dictionary
    name_onyen_dictionary = dictionary_initialization_chat_and_transcript(csv_file)
    hashed_first_names = ()
    hashed_last_names = ()
    hashed_onyen = ()
    fake_first_name_hashed_value_dictionary = {}
    fake_last_name_hashed_value_dictionary = {}
    fake_onyen_hashed_value_dictionary = {}
    with open(vtt_file) as f:
        lines = f.readlines()
    lines = np.array(lines)
    answer = []
    fake_name_answer = []
    hash_map = {}
    anti_hash_map = {}
    for rows in lines:
        # print("Original message: " + rows)
        if ":" in rows and rows.lstrip().rstrip() != "WEBVTT" and rows.lstrip().rstrip() != "" and rows[0] not in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] and rows[2] != ":":
            # initializing substrings
            sub = ": "

            # getting index of substrings
            idx = rows.index(sub)

            # length of substring 1 is added to
            # get string from next character
            name = rows[0: idx].lstrip().rstrip() # remove spaces
            if (len(name.split(" ")) < 2): # No first or last name
                first_name = name.split(" ")[0]
                last_name = ""
            else: # 这里存在一个问题，如果一个人的名字有不止一个空格（如middle name），将会导致first name与last name的选取出错
                first_name = name.split(" ")[0]
                last_name = name.split(" ")[1]

            # Using the dictionary to convert the names to corresponding onyens
            if name not in name_onyen_dictionary.keys():
                onyen = '*'+name+'*'
            else:
                onyen = name_onyen_dictionary[name]

            # sha-256 encryption
            if (first_name != ""):
                encoded_first = first_name.encode()
                result_first = hashlib.sha256(encoded_first)
                if (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # First time that encounters a first name
                    hashed_first_names = hashed_first_names + (result_first.hexdigest(),)
                    fake_first_name = first_name_array[0].replace("\n", "") # Always pick the first element
                    first_name_array = np.delete(first_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                    fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                    general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                elif (result_first.hexdigest() not in hashed_first_names) and (result_first.hexdigest() in general_first_name_hashmap): # NOT first time that encounters a first name
                    fake_first_name = general_first_name_hashmap[result_first.hexdigest()]
                    fake_first_name_hashed_value_dictionary.update({result_first.hexdigest():fake_first_name})
                elif (result_first.hexdigest() in hashed_first_names) and (result_first.hexdigest() not in general_first_name_hashmap): # NOT first time that encounters a first name
                    fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
                    general_first_name_hashmap.update({result_first.hexdigest():fake_first_name})
                else:
                    fake_first_name = fake_first_name_hashed_value_dictionary[result_first.hexdigest()]
            else: # First name is ""
                fake_first_name = ""

            if (last_name != ""):
                encoded_last = last_name.encode()
                result_last = hashlib.sha256(encoded_last)
                if (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # First time that encounters a last name
                    hashed_last_names = hashed_last_names + (result_last.hexdigest(),)
                    fake_last_name = last_name_array[0].replace("\n", "") # Always pick the first element
                    last_name_array = np.delete(last_name_array, 0) # Eliminate the name once it is used to prevent using one fake name for two different real names
                    fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                    general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                elif (result_last.hexdigest() not in hashed_last_names) and (result_last.hexdigest() in general_last_name_hashmap): # NOT first time that encounters a last name
                    fake_last_name = general_last_name_hashmap[result_last.hexdigest()]
                    fake_last_name_hashed_value_dictionary.update({result_last.hexdigest():fake_last_name})
                elif (result_last.hexdigest() in hashed_last_names) and (result_last.hexdigest() not in general_last_name_hashmap): # NOT first time that encounters a last name
                    fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
                    general_last_name_hashmap.update({result_last.hexdigest():fake_last_name})
                else:
                    fake_last_name = fake_last_name_hashed_value_dictionary[result_last.hexdigest()]
            else: # Last name is ""
                fake_last_name = ""

            if (onyen != ""):
                encoded_onyen = onyen.encode()
                result_onyen = hashlib.sha256(encoded_onyen)
                if (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # First time that encounters an onyen
                    hashed_onyen = hashed_onyen + (result_onyen.hexdigest(),)
                    if (fake_first_name != "") and (fake_last_name != ""):
                        fake_onyen = fake_first_name + fake_last_name # 这里暂时不处理同名同姓的不同人的问题，因为如果两人名字相同但onyen不同，在Zoom由name map到onyen时就会报错
                    elif (fake_first_name == "") and (fake_last_name != ""):
                        fake_onyen = fake_last_name + fake_last_name
                    else:
                        fake_onyen = fake_first_name + fake_first_name
                    fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                    general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                elif (result_onyen.hexdigest() not in hashed_onyen) and (result_onyen.hexdigest() in general_onyen_hashmap): # NOT first time that encounters an onyen
                    fake_onyen = general_onyen_hashmap[result_onyen.hexdigest()]
                    fake_onyen_hashed_value_dictionary.update({result_onyen.hexdigest():fake_onyen})
                elif (result_onyen.hexdigest() in hashed_onyen) and (result_onyen.hexdigest() not in general_onyen_hashmap): # NOT first time that encounters an onyen
                    fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
                    general_onyen_hashmap.update({result_onyen.hexdigest():fake_onyen})
                else:
                    fake_onyen = fake_onyen_hashed_value_dictionary[result_onyen.hexdigest()]
            else: # Onyen is ""
                fake_onyen = ""

            hashed_str = ""
            anti_hashed_str = ""
            name_onyen_substring = ""
            anti_name_onyen_substring = ""

            if (first_name != ""):
                name_onyen_substring = name_onyen_substring + str(result_first.hexdigest())
                anti_name_onyen_substring = anti_name_onyen_substring + fake_first_name
            if ((first_name != "") and (last_name != "")):
                name_onyen_substring = name_onyen_substring + " "
                anti_name_onyen_substring = anti_name_onyen_substring + " "
            if (last_name != ""):
                name_onyen_substring = name_onyen_substring + str(result_last.hexdigest())
                anti_name_onyen_substring = anti_name_onyen_substring + fake_last_name
            if (onyen != ""):
                name_onyen_substring = name_onyen_substring + " " + "(" + str(result_onyen.hexdigest()) + ")"
                anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + fake_onyen + ")"
            else:
                name_onyen_substring = name_onyen_substring + " " + "(" + ")"
                anti_name_onyen_substring = anti_name_onyen_substring + " " + "(" + ")"

            hashed_str = hashed_str + name_onyen_substring + rows[idx:]
            anti_hashed_str = anti_hashed_str + anti_name_onyen_substring + rows[idx:]

            # print("Hashed message: " + hashed_str)
            answer.append(hashed_str)
            fake_name_answer.append(anti_hashed_str)
            hash_map.update({name + " (" + onyen + ")":name_onyen_substring})
            if (first_name != "") and (last_name != "") and (onyen != ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (first_name == "") and (onyen == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " ()"})
            elif (last_name != "") and (onyen != ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " ()"})
            elif (first_name == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_last.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (last_name == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " (" + result_onyen.hexdigest() + ")"})
            elif (onyen == ""):
                anti_hash_map.update({anti_name_onyen_substring:result_first.hexdigest() + " " + result_last.hexdigest() + " ()"})
        else:
            answer.append(rows)
            fake_name_answer.append(rows)
    # Save the hashed txt file
    with open(vtt_file.split('.')[0] + "(hashed_audio_transcript).txt", 'w') as f:
            for line in answer:
                f.write(line)
    # Save the fake_named txt file
    with open(vtt_file.split('.')[0] + "(fake_name_transcript).txt", 'w') as f:
            for line in fake_name_answer:
                f.write(line)
    # Save the hashmap in json file
    with open(vtt_file.split('.')[0] +'(HashTable_audio_transcript).json', 'w') as fp:
        json.dump(hash_map, fp)
    # Save the anti_hashmap in json file
    with open(vtt_file.split('.')[0] +'(Anti_HashTable_transcript).json', 'w') as fp:
        json.dump(anti_hash_map, fp)
    # Update fake first names.csv
    delete_file = open(first_name_csv_file, "w")
    delete_file.truncate()
    with open(first_name_csv_file, 'w') as f:
            for line in first_name_array:
                f.write(line)
    # Update fake last names.csv
    delete_file = open(last_name_csv_file, "w")
    delete_file.truncate()
    with open(last_name_csv_file, 'w') as f:
            for line in last_name_array:
                f.write(line)
    # Update the general_anti_hashmap in json file
    delete_file = open(general_first_name_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_first_name_hashmap_json_file, 'w') as fp:
        json.dump(general_first_name_hashmap, fp)
    delete_file = open(general_last_name_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_last_name_hashmap_json_file, 'w') as fp:
        json.dump(general_last_name_hashmap, fp)
    delete_file = open(general_onyen_hashmap_json_file, "w")
    delete_file.truncate()
    with open(general_onyen_hashmap_json_file, 'w') as fp:
        json.dump(general_onyen_hashmap, fp)
    return

In [39]:
# unzip everything in a given path while keeping the original structure
def unzipdir(path):  
    # unzip everything in this directory first
    for file in os.listdir(path):
        #print(file)
        if (file[-4:]=='.zip'):
            filename=file[:-4] #remove .zip
            with zipfile.ZipFile(path+'\\'+file, 'r') as zip_ref:
                zip_ref.extractall(path+'\\'+filename)
            # remove empty folder or folder with only one subfolder
            if(len(os.listdir(path+'\\'+filename))==1):
                for i in os.listdir(path+'\\'+filename):
                    for j in os.listdir(path+'\\'+filename+'\\'+i):
                        shutil.move(path+'\\'+filename+'\\'+i+'\\'+j,path+'\\'+filename)
                    if not os.listdir(path+'\\'+filename+'\\'+i):
                        os.removedirs(path+'\\'+filename+'\\'+i)
            os.remove(path+'\\'+file) # remove all zip files
    for file2 in os.listdir(path):
        if (os.path.isdir(path+'\\'+file2)):
            unzipdir(path+'\\'+file2)
#unzipdir('C:\\Users\\PC-002\\OneDrive\\UNC\\Research\\Dewan\\CAT\\try1')

In [40]:
def anonymyzer(path,second_path,third_path,csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file):
    file_list = os.listdir(path)
    for i in file_list:
        if (('hashed' not in i) and ('HashTable' not in i) and ('fake_name' not in i) and ('.txt' in i)): #chat txt files
            extract_and_hash_chat(path+'\\'+i, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file)
            os.remove(path+'\\'+i)  #remove original file
        if (('hashed' not in i) and ('HashTable' not in i) and ('fake_name' not in i) and ('.vtt' in i)): # audio transcript vtt files
            extract_and_hash_audio_transcript(path+'\\'+i, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file)
            os.remove(path+'\\'+i)  #remove original file
    # move files in separate files
    file_list = os.listdir(path)
    try:
        os.makedirs(path+'\\'+'hashedChats') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(path+'\\'+'hashedTranscript') # create destination directory, if needed (similar to mkdir -p)
    except OSError:
        # The directory already existed, nothing to do
        pass
    try:
        os.makedirs(second_path+'\\'+'hashTables(Chat)') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(second_path+'\\'+'hashTables(Audio Transcript)') # create destination directory, if needed (similar to mkdir -p)
    except OSError:
        # The directory already existed, nothing to do
        pass
    try:
        os.makedirs(third_path+'\\'+'FakeNameChats') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(third_path+'\\'+'FakeNameTranscript') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(third_path+'\\'+'Anti_hashTables(Chat)') # create destination directory, if needed (similar to mkdir -p)
        os.makedirs(third_path+'\\'+'Anti_hashTables(Audio Transcript)') # create destination directory, if needed (similar to mkdir -p)
    except OSError:
        # The directory already existed, nothing to do
        pass

    for j in file_list:
        if ('(hashed_chat).txt' in j):
            shutil.move(path+'\\'+j,path+'\\'+'hashedChats')
        if ('(hashed_audio_transcript).txt' in j):
            shutil.move(path+'\\'+j,path+'\\'+'hashedTranscript')
        if ('(HashTable_chat).json' in j):
            shutil.move(path+'\\'+j,second_path+'\\'+'hashTables(Chat)')
        if ('(HashTable_audio_transcript).json' in j):
            shutil.move(path+'\\'+j,second_path+'\\'+'hashTables(Audio Transcript)')
        if ('(fake_name_chat).txt' in j):
            shutil.move(path+'\\'+j,third_path+'\\'+'FakeNameChats')
        if ('(fake_name_transcript).txt' in j):
            shutil.move(path+'\\'+j,third_path+'\\'+'FakeNameTranscript')
        if ('(Anti_HashTable_chat).json' in j):
            shutil.move(path+'\\'+j,third_path+'\\'+'Anti_hashTables(Chat)')
        if ('(Anti_HashTable_transcript).json' in j):
            shutil.move(path+'\\'+j,third_path+'\\'+'Anti_hashTables(Audio Transcript)')
    # recursively go through every subfolder
    for file2 in os.listdir(path):
        if (os.path.isdir(path+'\\'+file2) and ('hashedChats' not in file2) and ('hashedTranscript' not in file2) and ('hashTables(Chat)' not in file2) and ('hashTables(Audio Transcript)' not in file2) and ('FakeNameChats' not in file2) and ('FakeNameTranscript' not in file2) and ('Anti_hashTables(Chat)' not in file2) and ('Anti_hashTables(Audio Transcript)' not in file2)):
            anonymyzer(path+'\\'+file2 ,second_path+'\\'+file2,third_path+'\\'+file2, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file)

In [41]:
# remove empty folders in a given path
def removeempty(path):
    file_list = os.listdir(path)
    for i in file_list:
        if (os.path.isdir(path+'\\'+i)):
            if len(os.listdir(path+'\\'+i))==0:
                shutil.rmtree(path+'\\'+i)
            else:
                removeempty(path+'\\'+i)

In [42]:
def anonymyzer_driver(filename, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file):
    cwd = os.getcwd()
    path = cwd
    try:
        os.makedirs(cwd+'\\'+'hashTables') 
    except OSError:
            # The directory already existed, nothing to do
        pass
    # general unzip tha root file
    with zipfile.ZipFile(cwd+'\\'+filename+'.zip', 'r') as zip_ref:
        zip_ref.extractall(cwd+'\\'+filename)
    if(len(os.listdir(path+'\\'+filename))==1):
        for i in os.listdir(path+'\\'+filename):
            for j in os.listdir(path+'\\'+filename+'\\'+i):
                shutil.move(path+'\\'+filename+'\\'+i+'\\'+j,path+'\\'+filename)
            if not os.listdir(path+'\\'+filename+'\\'+i):
                os.removedirs(path+'\\'+filename+'\\'+i)
    # unzip all files
    unzipdir(cwd+'\\'+filename)
    # Create 3 enpty general anti_hashmap files
    with open(general_first_name_hashmap_json_file, 'w') as fp:
        json.dump({}, fp)
    with open(general_last_name_hashmap_json_file, 'w') as fp:
        json.dump({}, fp)
    with open(general_onyen_hashmap_json_file, 'w') as fp:
        json.dump({}, fp)
    # anonymize all files
    first_path = cwd+'\\'+filename
    second_path = cwd+'\\'+filename+'_hashTables'
    third_path = cwd+'\\'+filename+'_FakeName + Anti_hashTables'
    anonymyzer(first_path,second_path,third_path, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file)
    # remove all empty folders
    removeempty(path+'\\'+filename)
    removeempty(path+'\\'+filename+'_hashTables')

In [43]:
# run this cell to get the result
filename = 'try3'  # the name of zip file without .zip
csv_file = "grades.csv" #change the zip file name here WITH extension
first_name_csv_file = "fake first names.csv"
last_name_csv_file = "fake last names.csv"
general_first_name_hashmap_json_file = "General Anti_HashMap(first name).json"
general_last_name_hashmap_json_file = "General Anti_HashMap(last name).json"
general_onyen_hashmap_json_file = "General Anti_HashMap(onyen).json"
anonymyzer_driver(filename, csv_file, first_name_csv_file, last_name_csv_file, general_first_name_hashmap_json_file, general_last_name_hashmap_json_file, general_onyen_hashmap_json_file)